# Парсинг Ведомостей

Используем расширения для Chrome для удобного парсинга http://selectorgadget.com (+ используем fromstring из либы lxml)

Будем мультипоточно качать сразу по 50 статей с сайта одновременно (Pool)

In [8]:
import pandas as pd

In [9]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [10]:
import requests
from multiprocessing.dummy import Pool
from lxml.html import fromstring

def get_page(url):
    html = requests.get(url).text
    dom = fromstring(html)
    dom.make_links_absolute(url)
    return dom

def get_article_urls(url_date):
    pattern = '.b-article__title a'
    dom = get_page(url_date)
    return [t.attrib['href'] for t in dom.cssselect(pattern)]

def get_article_text(url):
    # todo: if 'video' or 'gallery' ... - return ''
    pattern_header = 'h1'
    patterns = ['.b-news-item__text_one p', '.b-social__layout-mutation p']
    
    try:
        dom = get_page(url)
    except:
        return None, None
    
    try:
        header = dom.cssselect(pattern_header)[0].text_content().strip()
    except:
        header = ""

    try:
        css_elements = flatten([dom.cssselect(pattern) for pattern in patterns])
        text = "\n\n".join([t.text_content().strip() for t in css_elements])
    except:
        text = ""

    return header, text

flatten = lambda l: [item for sublist in l for item in sublist]

In [11]:
url_date_template = 'https://www.vedomosti.ru/archive/{}/{}/{}'
url_dates = []
for y in [2016]:
    for m in range(1,13):
        for d in range(1,29):
            url_dates.append(url_date_template.format(y,m,d))

In [12]:
pool = Pool(50)

In [16]:
%%time
url_articles = pool.map(get_article_urls, url_dates)
url_articles = flatten(url_articles)
url_articles = [url for url in url_articles if url.split('/')[4] not in {'galleries','video','online'}]

CPU times: user 15.2 s, sys: 2.55 s, total: 17.8 s
Wall time: 1min 17s


вот сколько ссылок на статьи теперь у нас есть

In [17]:
url_articles

NameError: name 'url_articles' is not defined

In [14]:
len(url_articles)

NameError: name 'url_articles' is not defined

ограничимся пока 10000 статьями

In [ ]:
urls = url_articles[:10000]

In [ ]:
df = pd.DataFrame({"url": urls})

In [ ]:
df["topic"] = df.url.apply(lambda x: x.split('/')[3])
df["format"] = df.url.apply(lambda x: x.split('/')[4])

In [ ]:
df.head()

скачаем тексты статей

In [ ]:
%%time
headers_texts = pool.map(get_article_text, urls)

In [ ]:
pool.close()
pool.join()

объединим в датасет

In [ ]:
df["header"] = [t[0] for t in headers_texts]
df["text"] = [t[1] for t in headers_texts]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.to_pickle('vedomosti_archive.pkl')

длина заголовка

In [ ]:
df.header.apply(len).hist()

длина текста

In [ ]:
df.text.apply(len).hist()

длина текста, крупнее

In [ ]:
df[df.text.apply(len)<3000].text.apply(len).hist()

# Лемматизация слов

приведём все слова в текстах к начальной форме слова - так мы сильно сократим размерность пространства признаков

хоть pymorphy2 и достаточно быстрый, лучше его упаковывать в класс со своим кэшем (который хорошо бы ещё куда-то выгружать при окончании и загружать в начале работы, это можете доделать. Удобно для этих целей использовать json)

In [ ]:
import pymorphy2
from nltk.tokenize import word_tokenize

In [ ]:
class MorphProvider:
    def __init__(self):
        self.cache = {}
        self.morph = pymorphy2.MorphAnalyzer()
    
    def __call__(self, w):
        w = w.lower()
        cached = self.cache.get(w)
        if cached:
            return cached
        try:
            morphed = self.morph.parse(w)[0].normal_form
            self.cache[w] = morphed
            return morphed
        except:
            return None
    
    def morph_string(self, s):
        words = word_tokenize(s)
        return " ".join([self.__call__(w) for w in words])

In [ ]:
morph = MorphProvider()

In [ ]:
from tqdm import tqdm

In [ ]:
df["normalized"] = [morph.morph_string(t) for t in tqdm(df.text)]

In [ ]:
df.to_pickle('vedomosti_archive.pkl')

# Облако слов

*4 fun*

In [ ]:
from collections import Counter

In [ ]:
words = [w for w in " ".join(df.normalized).split() if len(w)>2]

In [ ]:
Counter(words).most_common()[:20]

In [ ]:
from wordcloud import WordCloud

def get_wordcloud(text,max_font_size=None):
    wordcloud = WordCloud(max_font_size=max_font_size).generate(text)
    plt.imshow(wordcloud)
    plt.axis("off")

In [ ]:
get_wordcloud(' '.join(words))

In [ ]:
get_wordcloud(' '.join(words), max_font_size=50)

-----